In [ ]:
from bs4 import BeautifulSoup
from urllib2 import urlopen
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import string
import dateutil.parser
import httplib
import time

%matplotlib inline

In [ ]:
BASE_URL = 'http://www.boxofficemojo.com/'

In [ ]:
def get_movie_links(section_url):
    html = urlopen(section_url).read()
    soup = BeautifulSoup(html,'lxml')
    a = soup.find('tr').find_all('a')
    a_regex = re.compile('s\/\?(.*?)"') 
    movie_links = []
    for line in a:
        result = re.search(a_regex, str(line))
        if result:
            movie_links.append(result.group(0))
    complete_movie_links = [(BASE_URL+"movie"+i[:-1]) for i in movie_links]
    return complete_movie_links

In [ ]:
alphabet = ["NUM"] + list(string.ascii_uppercase)
first_half = "http://www.boxofficemojo.com/movies/alphabetical.htm?letter="
second_half = "&p=.htm"
letter_links = [first_half + letter + second_half for letter in alphabet]

In [ ]:
def number_links(letter_url):
    html = urlopen(letter_url).read()
    soup = BeautifulSoup(html,'lxml')
    alpha = soup.find('div','alpha-nav-holder')
    number_link= [BASE_URL + a["href"] for a in alpha.findAll("a")]
    return number_link

In [ ]:
def get_movie_value(movie_link, field_name):
    soup = BeautifulSoup(requests.get(movie_link).text)
    '''Grab a value from boxofficemojo HTML
    
    Takes a string attribute of a movie on the page and
    returns the string in the next sibling object
    (the value for that attribute)
    or None if nothing is found.
    '''
    if field_name == 'title':
        title_string = soup.find(field_name).text
        title = title_string.split('(')[0].strip()
        return title
    else:
        obj = soup.find(text=re.compile(field_name))
        if not obj: 
            return None
        # this works for most of the values
        next_sibling = obj.findNextSibling()
        if next_sibling:
            return next_sibling.text 
        else:
            return None

In [ ]:
almost_all = [number_links(letter) for letter in letter_links]
almost_all2 = [item for sublist in almost_all for item in sublist]
real_all = almost_all2 +letter_links
all_movies = [get_movie_links(link) for link in real_all]
all_movies2 = [item for sublist in all_movies for item in sublist]

In [ ]:
len(all_movies2)

In [ ]:
b= 'http://www.boxofficemojo.com/movies/alphabetical.htm?letter=A&p=.htm'

In [ ]:
def to_date(datestring):
    try:
        date = dateutil.parser.parse(datestring)
        return date
    except:
        pass

def money_to_int(string):
    if string is not None:
        i = string.replace('$','').replace(',','').replace(' ','')
        if "million" in i:
            i = i.replace('million','')
            i = float(i) * 1000000
            i = int(i)
        return i
    else:
        return np.nan

In [ ]:
def all_movie_values(movie_link):
    runtime = get_movie_value(movie_link,'Runtime:')
    rating = get_movie_value(movie_link,'MPAA Rating:')
    production = get_movie_value(movie_link,'Production Budget:')
    distributor = get_movie_value(movie_link, 'Distributor:')
    release = get_movie_value(movie_link, 'Release Date:')
    domestic_total_gross = get_movie_value(movie_link, "Domestic Total Gross:")
    genre = get_movie_value(movie_link, "Genre:")
    title = get_movie_value(movie_link, "title")
    return {title:[money_to_int(domestic_total_gross),money_to_int(production),runtime,rating,to_date(release),genre]}

In [ ]:
movie_dict = {}
count = 0
for a in all_movies2:
    mov = all_movie_values(a)
    movie_dict.update(mov)
    time.sleep(.1)
    count += 1
    print count, mov

In [ ]:
m = []
for key, value in new5.items():
    m.append(value[:7])

In [ ]:
m1 = m[10035:]
m2 = m[:10035]
fixed_m = m1 + m2

In [ ]:
dtg_list = []
production_list = []
runtime_list = []
rating_list = []
date_list = []
genre_list = []
title_list = []
for i in fixed_m:
    try:
        dtg_list.append(int(i[0]))
    except:
        dtg_list.append(i[0])
    if type(i[1]) == int:
        production_list.append(i[1])
    else:
        production_list.append(np.nan)
    runtime_list.append(str(i[2]))
    rating_list.append(str(i[3]))
    date_list.append(i[4])
    genre_list.append(str(i[5]))
    title_list.append(str(i[6]))

In [ ]:
rows = zip(title_list, dtg_list, production_list, runtime_list, rating_list, date_list, genre_list)

In [ ]:
with open('output.csv', 'wb') as f:
    writer = csv.writer(f)
    for row in rows:
        writer.writerow(row)